<a href="https://colab.research.google.com/github/guk22/AIFFEL_quest_rs/blob/main/mnist_GCS_big_query.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install google-cloud-storage google-cloud-bigquery pandas tensorflow numpy
!pip install google-colab google-auth

In [ ]:
# 2. 라이브러리 임포트
import os
import io
import numpy as np
import pandas as pd
from google.colab import auth
from google.cloud import storage
from google.cloud import bigquery
from tensorflow.keras.datasets import mnist
import tensorflow as tf
import matplotlib.pyplot as plt
from PIL import Image

In [ ]:
# 3. Google Cloud 인증
auth.authenticate_user()

In [ ]:
# @markdown 프로젝트 ID와 버킷 이름을 입력하세요
project_id = "" # @param {type:"string"}
bucket_name = "" # @param {type:"string"}
assert project_id, "프로젝트 ID를 입력해주세요!"
assert bucket_name, "버킷 이름을 입력해주세요!"

In [ ]:
def create_bucket_if_not_exists(bucket_name, project_id):
    """Google Cloud Storage 버킷 생성"""
    storage_client = storage.Client(project=project_id)

    try:
        bucket = storage_client.get_bucket(bucket_name)
        print(f"버킷 {bucket_name}이 이미 존재합니다.")
    except Exception:
        print(f"버킷 {bucket_name}을 생성합니다...")
        bucket = storage_client.create_bucket(bucket_name, location="us")
        print(f"버킷이 생성되었습니다: {bucket.name}")

    return bucket

In [ ]:
def upload_image_to_gcs(image_array, bucket, image_path):
    """NumPy 배열을 PNG로 변환하여 GCS에 업로드"""
    img = Image.fromarray(image_array)
    img_byte_arr = io.BytesIO()
    img.save(img_byte_arr, format='PNG')
    img_byte_arr = img_byte_arr.getvalue()

    blob = bucket.blob(image_path)
    blob.upload_from_string(img_byte_arr, content_type='image/png')

    return f"gs://{bucket.name}/{image_path}"


In [ ]:
# 6. MNIST 데이터 로드 및 GCS 업로드
def process_mnist_data(bucket):
    """MNIST 데이터 처리 및 GCS 업로드"""
    print("MNIST 데이터 로드 중...")
    (x_train, y_train), (x_test, y_test) = mnist.load_data()

    data_records = []

    # 학습 데이터 처리
    # 학습 데이터 처리시 시간이 오래 걸리기 때문에 5000개만 사용하는 방식으로 코드를 완성시켜주세요.
    # 이번에는 image를 저장할때 url을 활용합니다. upload_image_to_gcs를 활용해서 코드를 만들어주세요.
    # 이번에는 data_records에 학습데이터와 테스트 데이터 모두 넣고 진행합니다.
    print("학습 데이터 처리 및 업로드 중...")
    # [[YOUR CODE]]

    # 테스트 데이터 처리 (샘플만)
    # 테스트 데이터도 학습 데이터와 똑같이 만들어주시되 1000개만 처리해주세요.

    return pd.DataFrame(data_records)


In [ ]:
def upload_to_bigquery(df, project_id, dataset_id, table_id):
    """데이터프레임을 BigQuery에 업로드"""
    # BigQuery 데이터셋 참조 객체 생성

    try:
        # 데이터셋 존재 여부 확인
    except Exception:
        # 데이터셋이 존재하지 않을 경우 새로 생성

        # 데이터셋의 지역을 US로 설정

        # 새로운 데이터셋 생성

    # 테이블 스키마 정의
    schema = [
        # 이미지 고유 식별자 (정수형)
        # 이미지 레이블/분류 값 (정수형)
        # 이미지 파일의 URL 주소 (문자열)
        # 데이터셋 종류 구분 (예: train, test, validation) (문자열)
    ]

    # 생성할 테이블 참조 객체 생성
    # 테이블 객체 생성 (정의된 스키마 적용)

    print("BigQuery에 데이터 업로드 중...")
    #데이터 로드를 위한 작업 설정 객체 생성

    # 데이터 로드 작업 실행
    job.result()
    print("업로드 완료!")

In [ ]:
def query_and_visualize(project_id, dataset_id, table_id):
    """BigQuery 데이터 쿼리 및 시각화"""
    client = bigquery.Client(project=project_id)

    # 레이블 별 이미지 수 조회
    print("\n레이블 별 이미지 수 분석 중...")
    count_query = f"""
        SELECT label, COUNT(*) as count
        FROM `{project_id}.{dataset_id}.{table_id}`
        GROUP BY label
        ORDER BY label
    """
    counts_df = client.query(count_query).to_dataframe()

    # 시각화
    plt.figure(figsize=(10, 5))
    plt.bar(counts_df['label'], counts_df['count'])
    plt.title('Number of Images per Label')
    plt.xlabel('Label')
    plt.ylabel('Count')
    plt.show()

    # 샘플 URL 출력
    print("\n각 레이블의 샘플 이미지 URL:")
    sample_query = f"""
        SELECT DISTINCT label,
               ARRAY_AGG(image_url LIMIT 1)[OFFSET(0)] as sample_url,
               COUNT(*) as count
        FROM `{project_id}.{dataset_id}.{table_id}`
        GROUP BY label
        ORDER BY label
    """
    samples_df = client.query(sample_query).to_dataframe()
    return samples_df

In [ ]:
# GCS 버킷 생성
bucket = create_bucket_if_not_exists(bucket_name, project_id)

In [ ]:
# MNIST 데이터 처리 및 GCS 업로드
mnist_df = process_mnist_data(bucket)

In [ ]:
# BigQuery에 메타데이터 업로드
dataset_id = 'mnist_dataset'
table_id = 'mnist_images_gcs'
upload_to_bigquery(mnist_df, project_id, dataset_id, table_id)

In [ ]:
# 데이터 분석 및 시각화
samples_df = query_and_visualize(project_id, dataset_id, table_id)
print("\n각 레이블별 샘플 이미지 정보:")
print(samples_df)

In [ ]:
# 특정 레이블의 이미지 URL 조회
query = f'''
    SELECT image_url
    FROM `{project_id}.{dataset_id}.{table_id}`
    WHERE label = 5
    LIMIT 5
'''
client = bigquery.Client(project=project_id)
df = client.query(query).to_dataframe()
print("숫자 5의 이미지 URL 샘플:")
print(df)